# Classification supervisée avec les modules KNeighborsClassifier et RandomForestClassifier de SCIKIT-Learn

In [13]:
#chargement des bibliothèques
import pandas as pd

In [14]:
#chargement des bibliothèque de machine learning
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [15]:
#chargement des données 
file_path = "C:/Users/Patrick Menan/Desktop/TP1/classification_supervisée/data_set.csv"
df = pd.read_csv(file_path)

In [208]:
df.head()

,X,Y,Classe
0,-0.697580,0.68494,0
1,-0.478690,0.63377,1
2,0.057028,0.91886,0
3,-0.593890,0.49488,0
4,0.229840,-0.41155,1


In [16]:
y = df.loc[:, 'Classe']
X = df.drop('Classe', axis = 'columns')
#y = df.loc[:, 'categorical_classe']
#X = df.drop('categorical_classe', axis = 'columns')

In [17]:
#separation apprentissage(train) et validation(valid)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.8)

In [211]:
#creation du model de classification par forêt aleatoire
model_forest_classifier = RandomForestClassifier(n_estimators=1000, random_state=1)

In [212]:
#entrainement du model
model_forest_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000, random_state=1)

In [213]:
#creation du model de classification par la methode des k-ppv (5-ppv dans notre cas ici)
model_neighbors_classifier = KNeighborsClassifier()

In [214]:
#entrainement du modèle
model_neighbors_classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [216]:
#evaluation de la performance forêt du modèle sur les données d'apprantissage
model_forest_classifier.score(X_train, y_train)

1.0

In [217]:
#evaluation de la performance du modèle k-ppv les données d'apprentissage
model_neighbors_classifier.score(X_train, y_train)

0.8297872340425532

In [221]:
#prediction avec le modèle Forêt
predict_forest=model_forest_classifier.predict(X_valid)
predict_forest

array([0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 0], dtype=int64)

In [223]:
#prediction avec le modèle K-ppv
predict_neighbors=model_neighbors_classifier.predict(X_valid)
predict_neighbors

array([0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 0], dtype=int64)

In [40]:
#fonction de prédiction d'un nouvel individu
def class_indiv(model, x, y):
    s = pd.DataFrame({'x' : [x], 'y': [y]})
    print(model.predict_proba(s))
    return model.predict(s)

""" 
def class_indiv(model, x, y):
    s = np.array([x, y]).reshape(1,2)
    print(model.predict_proba(s))
    return model.predict(s)
"""


' \ndef class_indiv(model, x, y):\n    s = np.array([x, y]).reshape(1,2)\n    print(model.predict_proba(s))\n    return model.predict(s)\n'

In [234]:
#prédiction d'un nouvel individu (x = 0.23, y = 0.2956) avec le model de classification par forêt aléatoire
class_indiv(model_forest_classifier, 0.23, 0.2956)

[[0.048 0.952]]


array([1], dtype=int64)

In [235]:
#prédiction d'un nouvel individu avec le modèle de classification par k-ppv, ayant les mêmes caratérisques que précédements
class_indiv(model_neighbors_classifier, 0.23, 0.2956)

[[0.2 0.8]]


array([1], dtype=int64)

# Choix des meilleurs hypers paramètres pour chacun des deux modèles

In [18]:
#utilisation de girdSearchCV
from sklearn.model_selection import GridSearchCV


In [26]:
#hyper paramètres à rechercher pour chaque modèle
my_n_estimators = [i for i in range(1,1000, 50)]
my_n_neighbors = [i for i in range(1,10)]
hyper_param_forest = {'n_estimators' : my_n_estimators}
hyper_param_neighbors = {'n_neighbors' : my_n_neighbors, 'metric': ['euclidean', 'manhattan']}
  

In [27]:
#mise en oeuvre
#on obtient dans chaque variable les meilleurs hyper paramètres pour chaque modèle
grid_neighbors = GridSearchCV(KNeighborsClassifier(), hyper_param_neighbors, cv=5)
grid_forest = GridSearchCV(RandomForestClassifier(), hyper_param_forest, cv=5)


In [33]:
#entrainement du modèle
grid_neighbors.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9]})

In [34]:
# score et paramètres du meilleur le modèle k-pp
print(grid_neighbors.best_score_)
print(grid_neighbors.best_params_)

0.775438596491228
{'metric': 'euclidean', 'n_neighbors': 7}


In [37]:
#sauvegarde du meilleur moldèle et estimatiion de sa performance sur les données de validation
model_neighbors = grid_neighbors.best_estimator_
model_neighbors.score(X_valid, y_valid)

0.6666666666666666

In [38]:
#entrainement du modèle forest
grid_forest.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'n_estimators': [1, 51, 101, 151, 201, 251, 301, 351,
                                          401, 451, 501, 551, 601, 651, 701,
                                          751, 801, 851, 901, 951]})

In [35]:
#score et paramètre du meilleur forest
print(grid_forest.best_score_)
print(grid_forest.best_params_)

0.7654970760233918
{'n_estimators': 151}


In [39]:
#sauvegarde du meilleur moldèle et estimatiion de sa performance sur les données de validation
model_forest = grid_forest.best_estimator_
model_forest.score(X_valid, y_valid)

0.875

# Conclusion

##Sur ce jeu de données, la performance obtenue avec le classifieur par forêt aléatoire est meileure que celle otenue par le classifieur K-ppp. Alors nous choisirons pour prédire un nouvel indivdu le modèle basé sur le classifieur par forêt aléatoire.

# Prédiction d'un nouvel individu 

In [41]:
#appel à la fonction class_indiv.
class_indiv(model_forest, 0.23, 0.2959)

[[0.19607843 0.80392157]]


array([1], dtype=int64)